<a href="https://colab.research.google.com/github/Yanis52/Skin-Cancer-Detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from pathlib import Path
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import pandas as pd
import seaborn as sns
import shutil
import os
from pathlib import Path
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split

In [3]:

drive.mount('/content/drive')
train_dir = Path("/content/drive/MyDrive/CBAM_project/augmented_data/Train")
# test_dir = Path("/content/drive/MyDrive/CBAM_project/augmented_data/Test")


Mounted at /content/drive


In [ ]:
dst_dir = "/content/dataset/Train"


if not os.path.exists(dst_dir):
    shutil.copytree(train_dir, dst_dir)
    print("✅ Données de train copiées.")
else:
  print("✅ Le dossier Train existe déjà dans /content/dataset.")

In [ ]:
# dst_train = "/content/dataset/Train"
# dst_test = "/content/dataset/Test"

# if not os.path.exists(dst_test):
#     shutil.copytree(test_dir, dst_test)
#     print("✅ Données de test copiées.")

**Définition du module CBAM**

In [ ]:
class AttentionBlock(tf.keras.layers.Layer):
    def __init__(self, filters, ratio=8, name="AttentionBlock", **kwargs):
        super().__init__(name=name, **kwargs)
        self.filters = filters
        self.ratio = ratio

    def build(self, input_shape):
        initializer = tf.keras.initializers.HeNormal()

        # Channel-wise attention
        self.avg_pool = layers.GlobalAveragePooling2D()
        self.max_pool = layers.GlobalMaxPooling2D()
        self.mlp = tf.keras.Sequential([
            layers.Dense(self.filters // (self.ratio*2), activation='relu', use_bias=False, kernel_initializer=initializer),
            layers.Dropout(0.2),
            layers.Dense(self.filters // self.ratio, activation='relu', use_bias=False, kernel_initializer=initializer),
            layers.Dropout(0.2),
            layers.Dense(self.filters, activation='sigmoid', use_bias=False, kernel_initializer=initializer)
        ])

        # Spatial attention block
        self.spatial = tf.keras.Sequential([
            layers.Conv2D(1, kernel_size=5, strides=1, padding='same', activation='sigmoid')

        ])



    def call(self, inputs,training=False):
        # Channel attention
        avg_out = self.avg_pool(inputs)
        max_out = self.max_pool(inputs)
        avg_weight = self.mlp(avg_out)
        max_weight = self.mlp(max_out)
        scale = tf.nn.sigmoid(avg_weight + max_weight)
        scale = tf.reshape(scale, [-1, 1, 1, self.filters])
        x = inputs * scale


        # Spatial attention
        avg_map = tf.reduce_mean(x, axis=-1, keepdims=True)
        max_map = tf.reduce_max(x, axis=-1, keepdims=True)
        combined = tf.concat([avg_map, max_map], axis=-1)
        attn_map = self.spatial(combined)

        return x * attn_map


Construction du **modèle**

In [ ]:
def build_cbam_model(input_shape=(128, 128, 3), num_classes=9):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Rescaling(1./255)(inputs)

    for i, filters in enumerate([64, 128, 256, 512]):
        x = layers.Conv2D(filters, 3, padding="same", use_bias=False, name=f"conv_{i}")(x)
        x = layers.BatchNormalization(name=f"bn_{i}")(x)
        x = layers.LeakyReLU(name=f"leakyrelu_{i}")(x)
        x = AttentionBlock(filters, name=f"cbam_block_{i}")(x)
        x = layers.MaxPooling2D(name=f"maxpool_{i}")(x)

    x = layers.GlobalAveragePooling2D(name="gap")(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(512)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax', name="output")(x)

    return tf.keras.Model(inputs, outputs, name="CBAM_Model")


Chargement du dataset depuis les images augmentées

In [ ]:
# img_height = 128
# img_width = 128
# batch_size = 32
# num_classes = 9



# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     train_dir,
#     seed=123,
#     image_size=(img_height, img_width),
#     batch_size=batch_size
# )

# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     test_dir,
#     seed=123,
#     image_size=(img_height, img_width),
#     batch_size=batch_size
# )


In [ ]:
# split avec tensorflow

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size,
       shuffle=True
       )
       class_names = train_ds.class_names

       val_ds = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        seed=123,
        validation_split=0.2,
        subset="validation",
        image_size=(img_height, img_width),
        batch_size=batch_size,
        huffle=False
 )

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


In [ ]:
print(train_ds.class_names)

In [ ]:
train_labels = []
for images, labels in train_ds:
    train_labels.extend(labels.numpy())

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Dictionnaire format {classe: poids}
class_weights = dict(enumerate(class_weights))

compilation

In [ ]:
model = build_cbam_model(input_shape=(img_height, img_width, 3), num_classes=num_classes)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


callbacks

In [ ]:
checkpoint_path = "/content/drive/MyDrive/CBAM_project/models/model8.keras"
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    # tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
]


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=callbacks,
    class_weight=class_weights
)


In [ ]:
import os

model.save("/content/drive/MyDrive/CBAM_project/models/model_test_full.keras")
size_mb = os.path.getsize("/content/drive/MyDrive/CBAM_project/models/model_test_full.keras") / (1024*1024)
print(f"Modèle sauvegardé, taille : {size_mb:.1f} Mo")


Évolution de la précision et de la perte par epoch

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, acc, label='Train accuracy')
plt.plot(epochs, val_acc, label='Val accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label='Train loss')
plt.plot(epochs, val_loss, label='Val loss')
plt.title('Loss over epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
from pathlib import Path

train_dir = Path("/content/drive/MyDrive/CBAM_project/augmented_data/Train")
for c in sorted(train_dir.iterdir()):
    if c.is_dir():
            count = len(list(c.glob("*.jpg")))
            print(f"{c.name}: {count} images")


In [ ]:
unique_true = np.unique(y_true)
print("Classes présentes dans y_true :", unique_true)
print("Classes attendues :", list(range(len(class_names))))
print("Noms des classes :", [class_names[i] for i in unique_true])


matrice de confusion

In [ ]:


y_true = []
y_pred = []

for images, labels in val_ds:
  preds = model.predict(images, verbose=0)
  y_true.extend(labels.numpy())
  y_pred.extend(np.argmax(preds, axis=1))

            # Matrice de confusion
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
fig, ax = plt.subplots(figsize=(10, 8))
disp.plot(ax=ax, xticks_rotation=90, cmap="Blues")
plt.title("Matrice de confusion")
plt.show()




 Distribution des prédictions (par classe)

In [ ]:


# Distribution des prédictions
df_pred = pd.DataFrame({'true': y_true, 'pred': y_pred})
df_pred['true_class'] = df_pred['true'].map(lambda i: class_names[i])
df_pred['pred_class'] = df_pred['pred'].map(lambda i: class_names[i])

plt.figure(figsize=(12, 5))
sns.countplot(x='pred_class', data=df_pred, order=class_names)
plt.title("Distribution des prédictions par classe")
plt.xticks(rotation=45)
plt.show()


Graphe des erreurs par classe

In [ ]:
# Pourcentage d'erreurs par classe
error_by_class = (df_pred['true'] != df_pred['pred']).groupby(df_pred['true_class']).mean()

error_by_class.plot(kind='bar', figsize=(10, 5), color='salmon')
plt.title("Taux d'erreur par classe")
plt.ylabel("Taux d'erreur")
plt.xticks(rotation=45)
plt.ylim(0, 1)
plt.show()
